Get all TLV streets

In [1]:
import geopandas as gpd
from shapely.geometry import LineString
import pandas as pd

import json
data = None
with open("./streets/streets.json", 'r') as f:
    data = json.load(f)

data = data["features"]

geometries = [LineString(feature['geometry']['paths'][0]) for feature in data]
attributes = [feature['attributes'] for feature in data]

streets = gpd.GeoDataFrame(attributes, geometry=geometries)

Get all TLV street lights

In [2]:
from shapely.geometry import Point

all_lights = pd.read_csv("/Users/omrilevy/Desktop/safe-city/all_street_lights_coords.csv")
# Create a geometry column from x and y coordinates
geometry = [Point(xy) for xy in zip(all_lights['x'], all_lights['y'])]

# Create a geopandas GeoDataFrame
gdf = gpd.GeoDataFrame(all_lights, geometry=geometry)


In [3]:
gdf.head()

,x,y,geometry
0,34.786088,32.118293,POINT (34.78609 32.11829)
1,34.764394,32.064291,POINT (34.76439 32.06429)
2,34.813693,32.123650,POINT (34.81369 32.12365)
3,34.802815,32.066177,POINT (34.80282 32.06618)
4,34.803828,32.101972,POINT (34.80383 32.10197)


In [4]:
streets.head()

,oid_rechov,k_rechov,t_rechov,shem_angli,ms_lamas,t_sug,k_kivun,UniqueId,shem_arvit,geometry
0,1,915,הרוגי מלכות,HARUGEY MALKHOT,336,רחוב,0,507-10001,قتل مملكة,"LINESTRING (34.83219 32.10998, 34.83250 32.10984)"
1,2,0,0,UKNOWN,0,רחוב,3,507-10002,None,"LINESTRING (34.76948 32.08689, 34.77027 32.08679)"
2,3,265,אמסטרדם,AMSTERDAM,516,רחוב,1,507-10003,أمستردام,"LINESTRING (34.77889 32.08664, 34.77954 32.08648)"
3,4,644,אלון יגאל,YIG'AL ALLON,2524,רחוב,0,507-10004,ألون ييغال,"LINESTRING (34.79662 32.07734, 34.79675 32.07755)"
4,5,634,מרגולין,MARGOLIN,2649,רחוב,1,507-10005,مارغولين,"LINESTRING (34.79370 32.05846, 34.79455 32.05844)"


Match each street light with the closest street

In [5]:
import geopandas as gpd
from shapely.ops import nearest_points

# Create a new column in the lights GeoDataFrame to store the closest street
gdf['k_rechov'] = None
gdf['t_rechov'] = None
gdf["english_name"] = None

count = 0
for point in gdf.itertuples():
    # Iterate over each light and find the closest street
    light_point = point.geometry
    closest_street = streets.loc[streets["geometry"].distance(light_point).idxmin()]
    gdf.at[point.Index, 'k_rechov'] = closest_street["k_rechov"]
    gdf.at[point.Index, 't_rechov'] = closest_street["t_rechov"]
    gdf.at[point.Index, 'english_name'] = closest_street["shem_angli"]

  
gdf.head()

# # Save the result as a CSV
gdf.to_csv("closest_streets.csv", index=False)


In [6]:
gdf.head()

,x,y,geometry,k_rechov,t_rechov,english_name
0,34.786088,32.118293,POINT (34.78609 32.11829),2222,ארצי יצחק,ARTZI ITZHAK
1,34.764394,32.064291,POINT (34.76439 32.06429),453,"איל""ן",ILAN
2,34.813693,32.123650,POINT (34.81369 32.12365),852,מסקין אהרון,MESKIN
3,34.802815,32.066177,POINT (34.80282 32.06618),635,לה גוארדיה,LA-GUARDIA
4,34.803828,32.101972,POINT (34.80383 32.10197),1250,קוסובסקי,KOSOVSKY


In [8]:
# gdf["english_name"].value_counts().to_csv("./num_street_lights_in_street.csv")


In [11]:
num_lights = pd.read_csv("./num_street_lights_in_street.csv")

In [12]:
num_lights.head()

,english_name,count
0,OR HAHAYIM,1419
1,BEGIN MENAHEM,1298
2,HAGIMNASYA HA'IVRIT,1069
3,ILAN,920
4,HATNUPHA,859


In [13]:
target_crs = 'EPSG:32636'  # UTM Zone 36N example

# Set the CRS for the GeoDataFrame
streets.crs = 'EPSG:4326'  # WGS84 CRS

# Define the target CRS that uses meters as the unit, such as UTM
target_crs = 'EPSG:32636'  # UTM Zone 36N example

# Reproject the GeoDataFrame to the target CRS
streets_reprojected = streets.to_crs(target_crs)
streets_reprojected["length"] = streets_reprojected['geometry'].length


In [14]:
streets_reprojected.head()

,oid_rechov,k_rechov,t_rechov,shem_angli,ms_lamas,t_sug,k_kivun,UniqueId,shem_arvit,geometry,length
0,1,915,הרוגי מלכות,HARUGEY MALKHOT,336,רחוב,0,507-10001,قتل مملكة,"LINESTRING (672865.880 3554095.253, 672895.215...",32.805669
1,2,0,0,UKNOWN,0,רחוב,3,507-10002,None,"LINESTRING (666990.499 3551436.940, 667065.337...",75.559074
2,3,265,אמסטרדם,AMSTERDAM,516,רחוב,1,507-10003,أمستردام,"LINESTRING (667879.711 3551424.163, 667940.741...",63.283575
3,4,644,אלון יגאל,YIG'AL ALLON,2524,רחוב,0,507-10004,ألون ييغال,"LINESTRING (669570.036 3550420.535, 669581.404...",25.979677
4,5,634,מרגולין,MARGOLIN,2649,רחוב,1,507-10005,مارغولين,"LINESTRING (669329.152 3548322.758, 669409.403...",80.262678


In [17]:
num_lights.columns = ["shem_angli", "count"]

In [18]:
num_lights.head()

,shem_angli,count
0,OR HAHAYIM,1419
1,BEGIN MENAHEM,1298
2,HAGIMNASYA HA'IVRIT,1069
3,ILAN,920
4,HATNUPHA,859


In [19]:
streets_reprojected.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   oid_rechov  1000 non-null   int64   
 1   k_rechov    1000 non-null   int64   
 2   t_rechov    1000 non-null   object  
 3   shem_angli  1000 non-null   object  
 4   ms_lamas    1000 non-null   int64   
 5   t_sug       1000 non-null   object  
 6   k_kivun     1000 non-null   int64   
 7   UniqueId    1000 non-null   object  
 8   shem_arvit  995 non-null    object  
 9   geometry    1000 non-null   geometry
 10  length      1000 non-null   float64 
dtypes: float64(1), geometry(1), int64(4), object(5)
memory usage: 86.1+ KB


In [20]:
updated_streets = streets_reprojected.merge(num_lights, on='shem_angli')
df = updated_streets.drop('geometry', axis=1)
df["lights_per_meter"] = df["count"] / df["length"]

In [21]:
df = updated_streets.drop('geometry', axis=1)
df["lights_per_meter"] = df["count"] / df["length"]

In [22]:
grouped = df.groupby(by="shem_angli").sum().reset_index()
grouped['normalized_lights_per_meter'] = (grouped['lights_per_meter'] - grouped['lights_per_meter'].min()) / (grouped['lights_per_meter'].max() - grouped['lights_per_meter'].min())

/var/folders/bg/t3kv0jz57dd_0dlwft3x0b3r0000gn/T/ipykernel_28715/3031813574.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = df.groupby(by="shem_angli").sum().reset_index()


In [28]:
grouped['normalized_lights_per_meter'] = (grouped['lights_per_meter'] - grouped['lights_per_meter'].min()) / (grouped['lights_per_meter'].max() - grouped['lights_per_meter'].min())
grouped["shem_angli"] = grouped["shem_angli"].str.lower()

In [29]:
grouped.to_csv("final.csv")

In [30]:
grouped.head()

,shem_angli,oid_rechov,k_rechov,ms_lamas,k_kivun,length,count,lights_per_meter,normalized_lights_per_meter
0,"""hevra hadasha""",1724,1142,1878,2,125.447962,16,0.256187,0.001535
1,1115,47,1115,4049,2,96.305900,437,4.537624,0.028126
2,1221,378,1221,4103,0,52.293335,2,0.038246,0.000182
3,1279,733,1279,946,3,58.605763,4,0.068253,0.000368
4,2040,236,2040,3130,0,613.798959,228,0.371457,0.002251


In [31]:
grouped["shem_angli"].str.lower()

0      "hevra hadasha"
1                 1115
2                 1221
3                 1279
4                 2040
            ...       
399    zikhron ya'akov
400              ziman
401           zirelson
402                ziv
403         zola emile
Name: shem_angli, Length: 404, dtype: object